# Project 4: Finetuning and Prompting

In this project, you will first learn how to use Huggingface's Transformers library to load large language models. Next, we will generate text from these models. Finally, we will work with a small text-to-SQL dataset, where the input is a natural language query and the output is a SQL query that can be executed against a database. You will explore (1) finetuning a pretrained language model, and (2) prompting the pretrained language model with examples. 

This project will be more open ended than the previous projects. We expect you to learn how to use the huggingface and torch documentation.

## Setup

First we install and import the required dependencies. These include:
* `torch` for modeling and training
* `transformers` for pre-trained models
* `datasets` from huggingface to load existing datasets.

In [1]:
%%capture
!pip install transformers
!pip install datasets
!pip install sentence-transformers

# Standard library imports
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM

Before proceeding, let's verify that we're connected to a GPU runtime and that `torch` can detect the GPU.
We'll define a variable `device` here to use throughout the code so that we can easily change to run on CPU for debugging.

In [2]:
assert torch.cuda.is_available()
device = torch.device("cuda")
print("Using device:", device)

Using device: cuda


## Loading Model

We will use GPT-2 Medium for this project. This includes both the GPT-2 tokenizer and the GPT-2 model weights itself. If you want to learn more about this model, you can read the GPT-2 paper https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf.

Let's first load the tokenizer for the GPT-2 medium model. You can find how to do this by reading the documentation for AutoTokenzier in transformers, and finding the GPT-2 model of ~345 million params in there.

In [3]:
from transformers import AutoTokenizer
# Your code here
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium")

tokenizer.pad_token = tokenizer.eos_token # convenient for padding later

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Let's tokenize and detokenize some text from this model.

In [4]:
print(tokenizer.encode('Hello world'))
print(tokenizer.decode(tokenizer.encode('Hello world')))
print(tokenizer.encode("Hola, cómo estás😍"))

[15496, 995]
Hello world
[39, 5708, 11, 269, 10205, 5908, 1556, 40138, 47249, 235]


Now let's load the GPT-2 Medium model. Make sure you also put the model onto the GPU.

In [63]:
from transformers import AutoModelForCausalLM
# Your code here
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2-medium").to(device)

## Generate From the Model

Now let's generate some text from the model to test its LM capabilities. Let's first generate one output of length 50 tokens using greedy decoding (temperature = 0), which should get us some text with high likelihood under the model. When generating text, you can condition on phrases such as "The coolest thing in NLP right now is". Find the relevant function and arguments to use for generating text using the Huggingface documentation.

Hint: you may find https://huggingface.co/docs/transformers/main_classes/text_generation to be useful for learning about generating from LMs.

In [44]:
inputs = tokenizer("The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
sample_output = gpt2_model.generate(inputs, do_sample=True, temperature=0.7, top_k=2, max_new_tokens=50)[0]

print("{}".format(tokenizer.decode(sample_output, skip_special_tokens=True)))

The coolest thing right now in NLP is the ability to create a deep understanding of the human brain. We're not just talking about a few hundred words here, we're talking about a whole brain. We're not talking about a few hundred words, we're talking about a whole brain.


Now let's generate 10 pieces of random text of length 50 tokens from the model using random sampling with temperature set to 0.7. This will allow the text to be somewhat higher in diversity (random sampling) while maintaining reasonable quality (temperature < 1). 

In [45]:
inputs = tokenizer("The coolest thing right now in NLP is", return_tensors="pt").input_ids.cuda()
# Your code here
sample_outputs = gpt2_model.generate(inputs, num_return_sequences=10, do_sample=True, temperature=0.7, top_k=2, max_new_tokens=50)

for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0: The coolest thing right now in NLP is the ability to create a machine learning model that can predict the future. It's not just about predicting the future, it's about predicting what's going to happen in the future.

I'm a big believer in machine learning, but it's
1: The coolest thing right now in NLP is the ability to use machine learning to predict the future.

The most interesting part of this is that we can predict what the future will be.

We know what the future is going to be, but we don't know what the future
2: The coolest thing right now in NLP is that it's not just about the data. It's about the people who are doing it, and how they're doing it. It's about the people who are using it, and how they're using it.

I'm a big believer
3: The coolest thing right now in NLP is the ability to use machine learning to predict the future. We can use machine learning to predict the future of a company, a product, or a person. We have a lot more data than we have ever had before. We

## Text-to-SQL Task Setup

First, let's download the data of text-to-SQL pairs and the database against which we'll execute queries to retrieve answers.

The code below initializes the database and does some initial preprocessing data preprocessing + splitting for you.

In [46]:
%%capture
!wget https://github.com/jkkummerfeld/text2sql-data/raw/master/data/geography.json
!wget https://github.com/jkkummerfeld/text2sql-data/raw/master/data/geography-db.sql

import re
import sqlite3
import json
from copy import deepcopy

DATABASE_NAME = 'geo.db'
SQL_FILENAME = 'geography-db.sql'
DATASET_FILENAME = 'geography.json'

with open(SQL_FILENAME, 'r') as file:
    sql_script = file.read()
    sql_script = re.sub(r"\s*ENGINE=[^ ]+","", sql_script)
    sql_script = re.sub(r"\s*DEFAULT CHARSET=[^ ;]+","", sql_script)
    sql_script = re.sub(r"\s*LOCK TABLES `[^`]+` WRITE;","", sql_script)  # remove LOCK TABLES
    sql_script = re.sub(r"\s*UNLOCK TABLES;","", sql_script)  # remove UNLOCK TABLES
    sql_script = sql_script.replace('`', '')  # remove backticks

# Connect to the SQLite database (this will create the file if it doesn't exist)
connection = sqlite3.connect(DATABASE_NAME)
print(sql_script)

connection.executescript(sql_script)
connection.commit()
connection.close()

connection = sqlite3.connect(DATABASE_NAME)
cursor = connection.cursor()
with open(DATASET_FILENAME, 'r') as file:
    dataset = json.load(file)

splits = {'train': [], 'dev': [], 'test': []}
for query_type in dataset:
    for example in query_type['sentences']:
        split = example['question-split']
        example['question'] = example['text']
        for key, value in example['variables'].items():
            example['question'] = example['question'].replace(key, value)
        example['sql'] = deepcopy(query_type['sql'])
        example['sql'] = example['sql'][0]
        for key, value in example['variables'].items():
            example['sql'] = example['sql'].replace(key, value)
        try:
            cursor.execute(example['sql'])
        except:
            continue
        example['db_answer'] = cursor.fetchall()
        del example['text']
        del example['variables']
        splits[split].append(example)

We also provide a function you can use to query the database:

In [47]:
def query_db(sql):
    connection = sqlite3.connect(DATABASE_NAME)
    cursor = connection.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    connection.close()
    return result

This dataset is pretty small:

In [48]:
print('Train set size:', len(splits['train']))
print('Dev set size:', len(splits['dev']))
print('Test set size:', len(splits['test']))

Train set size: 547
Dev set size: 48
Test set size: 277


Let's inspect an example from the training dataset:

In [49]:
splits['train'][0]

{'question-split': 'train',
 'question': 'what is the biggest city in nebraska',
 'sql': 'SELECT CITYalias0.CITY_NAME FROM CITY AS CITYalias0 WHERE CITYalias0.POPULATION = ( SELECT MAX( CITYalias1.POPULATION ) FROM CITY AS CITYalias1 WHERE CITYalias1.STATE_NAME = "nebraska" ) AND CITYalias0.STATE_NAME = "nebraska" ;',
 'db_answer': [('omaha',)]}

Note that the `db_answer` is the result of executing the given SQL output against the database:

In [50]:
query_db(splits['train'][0]['sql'])

[('omaha',)]

Let's check how well our language model does on this text-to-SQL task out of the box. You can just use greedy decoding. 

In [51]:
prompt = "Write a SQL query based on the following question.\n\nQuestion: {input}\n\nSQL:"

In [52]:
# Your code here. Generate from the model using greedy decoding with the above prompt
inputs = tokenizer(prompt, return_tensors="pt").input_ids.cuda()
#output = gpt2_model.generate(inputs, do_sample=True, temperature=0.7, top_k=2, max_new_tokens=50)[0]
output = gpt2_model.generate(inputs, max_length=50, temperature=0, num_return_sequences=1)[0]
predicted_sql = tokenizer.decode(output, skip_special_tokens=True) 
print(predicted_sql)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Write a SQL query based on the following question.

Question: {input}

SQL: SELECT * FROM {input}

Output:

SELECT * FROM {input}

Question: {input}

SQL:


You should get something that looks kind of like a SQL query, but it probably won't match the correct output, and in fact it most likely won't even execute without crashing when you try to query the database (you'll see a syntax error below).

In [53]:
try:
    query_db(predicted_sql)
    print('success!')
except:
    print('failed to execute!')

failed to execute!


Let's confirm quantitatively that the model doesn't work well out-of-the-box by running on the dev dataset (`splits['dev']`).

In [54]:
@torch.no_grad()
def predict_greedy(model, data, max_new_tokens=128):
    """
    Return the model's greedy text-to-sql predictions on the given data split.
    The maximum number of new tokens generated (NOT including tokens in the prompt) should be equal to max_new_tokens.
    For speed, you should batch the generation. The tokenizer can handle multiple inputs simultaneously,
    but you'll need to tell it to pad using padding=True, and you may also need to set tokenizer.padding_side='left'.
    Hint: as a postprocessing step after you're done, you may need to cut off the output at the first appearance of '\n' if the output is continuing past the end of the SQL.
    """
    questions = [d['question'] for d in data]
    predicted_sqls = []
    # Your code here
    batch_size=8
    model=model.to(device)
    model.eval()
    tokenizer.padding_side='left'
    PROMPT = "Question: {question}\n\nSQL:"
    questions=[PROMPT.format(question=i) for i in questions]
    for i in range(0, len(questions), batch_size):
        batch_questions = questions[i:i + batch_size]
        inputs = tokenizer(batch_questions, return_tensors="pt", padding=True).input_ids.cuda()
        output = model.generate(inputs, max_length=max_new_tokens, num_return_sequences=1, temperature=0,do_sample=False)
        generated_sqls = tokenizer.batch_decode(output, skip_special_tokens=True, max_length=max_new_tokens)
        
        for i in range(len(generated_sqls)):
            if '\n\n' in generated_sqls[i]:
                components = generated_sqls[i].split('\n\n')
                predicted_sqls.append(components[1].replace("SQL: ", ""))
    return predicted_sqls # list of strings containing SQL predictions for each question in the data

In [55]:
def check_execution_accuracy(predictions, data):
    assert len(predictions) == len(data)
    correct = 0
    for p, d in zip(predictions, data):
        try:
            if query_db(p) == d['db_answer']:
                correct += 1
        except: # failed to execute
            pass
    return correct / len(predictions)

In [56]:
predictions = predict_greedy(gpt2_model, splits['dev'])
print('example prediction:', predictions[0])
print('initial execution acc', check_execution_accuracy(predictions, splits['dev']))

example prediction: SQL:
initial execution acc 0.0


You will probably observe an accuracy around 0-2%. (It may be hard to verify if your `predict_greedy` function is correct at this stage, because the expected accuracy is so low, but you will reuse it later with an improved model, at which point it will be more obvious if your implementation is correct.)

## Model Finetuning

Now let's prepare our dataset for finetuning (i.e., training our pretrained language model on this text-to-SQL training set). For each element in the dataset, it should have a text prompt and then the SQL output, similar to above. Your job is to fill in the labels field below. This field sets the labels to use for training during the language modeling task.

For the labels, we only want to train the model to output the text after the word "SQL:". This is because in the prompt, everything before the word "SQL:" will also be provided to the model as input. Hint: use -100 as the label for tokens you do not want to train on. Hint 2: When doing LM training, the labels are the same as the input tokens, except shifted to the left by one. You should check whether Huggingface is already doing the shifting, or whether you need to do the shifting yourself.

One thing to be careful of with all LMs is to make sure there are not extra spaces. So, the text should be formatted as like "SQL: {sql output}" not "SQL: {sql output} ". 

In [57]:
class Text2SQLDataset(Dataset):
    PROMPT = "Write a SQL query based on the following question.\n\nQuestion: {question}\n\nSQL: {sql}"

    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        tokenizer.padding_side = 'right'

        self.input_ids = []
        self.attn_masks = []
        self.labels = []

        training_texts = []
        for example in self.data:
            training_text = Text2SQLDataset.PROMPT.format(question=example['question'], sql=example['sql']) + "<|endoftext|>" # include the end token so model knows when to stop!
            training_texts.append(training_text)
        encodings_dict = self.tokenizer(training_texts, padding=True, truncation=True)
        for i, (example, training_text) in enumerate(zip(data, training_texts)):
            self.input_ids.append(torch.tensor(encodings_dict['input_ids'][i]))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][i]))
            # Your code here
            input_tokens = self.tokenizer.tokenize(training_text)
            sql_start = input_tokens.index('SQL') 
            label = encodings_dict['input_ids'][i]
            label[:sql_start]=[-100]*sql_start
            self.labels.append(torch.tensor(label))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {'input_ids':(self.input_ids[idx]),
                'attn_masks': (self.attn_masks[idx]),
                'labels': (self.labels[idx])}
        #return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

In [58]:
train_dataset = Text2SQLDataset(splits['train'], tokenizer)
dev_dataset = Text2SQLDataset(splits['dev'], tokenizer)
test_dataset = Text2SQLDataset(splits['test'], tokenizer)

Now we can use the Huggingface Trainer to finetune GPT-2 Medium on this dataset. This abstracts away all of the details of training. Setup the training arguments to perform 3 epochs of training on this dataset, use a per-device batch size of 2 with gradient accumulation set to 8, use 30 warmup steps, a weight decay of 0.05. Set the eval batch size to be 8. Save a checkpoint after 100 steps. Set fp16 to True. Save the checkpoint in a specific output_dir so you can load it later. Hint: if it tries to launch Wandb, you may add the argument report_to="none".

In [64]:
from transformers import TrainingArguments, DataCollatorWithPadding

# Define the output directory to save checkpoints
output_dir = "/kaggle/working/checkpoint"


# Training arguments
# 设置训练参数
training_args = TrainingArguments(
    output_dir="./output",              # 保存模型的路径
    evaluation_strategy="steps",        # 每隔若干步进行评估
    num_train_epochs=3,                 # 训练3个epoch
    per_device_train_batch_size=2,      # 每个设备的训练批次大小为2
    gradient_accumulation_steps=8,      # 梯度累积步数为8
    warmup_steps=30,                    # 30个warmup步数
    weight_decay=0.05,                  # 权重衰减0.05
    save_steps=100,                     # 每100步保存一个检查点
    fp16=True,                          # 使用16位浮点数进行训练
    report_to="none",                   # 禁用Wandb报告
    logging_dir='./logs',               # 日志保存目录
    logging_steps=10                    # 每10步记录一次日志
)

# print(train_dataset.__getitem__(0))
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)

trainer=Trainer(
    model=gpt2_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=dev_dataset
)

trainer.train()
# model.save_pretrained(output_dir)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
10,3.781600,0.539835
20,0.374400,0.106600
30,0.080500,0.045324
40,0.044700,0.032861
50,0.033200,0.029417


TrainOutput(global_step=51, training_loss=0.8463447819664782, metrics={'train_runtime': 349.5598, 'train_samples_per_second': 4.694, 'train_steps_per_second': 0.146, 'total_flos': 892894929469440.0, 'train_loss': 0.8463447819664782, 'epoch': 2.978102189781022})

Reload the final saved version of the model below. You may need to delete the previously loaded model if you run out of GPU memory.

In [65]:
del gpt2_model
# Your code here
finetuned_model = AutoModelForCausalLM.from_pretrained('/kaggle/working/checkpoint/checkpoint-51')

Let's check our finetuned model's performance.

In [66]:
finetuned_predictions = predict_greedy(finetuned_model, splits['test'])
print('finetuned execution acc:', check_execution_accuracy(finetuned_predictions, splits['test']))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


finetuned execution acc: 0.27075812274368233


You should achieve an accuracy of roughly 50% using the suggested training hyperparameters; we will check >40% in the autograder.

Save your predictions.

In [ ]:
def save_predictions(predictions, filename):
    with open(filename, 'w') as f:
        f.write('\n'.join(predictions))

save_predictions(finetuned_predictions, 'finetuned_predictions.txt')

Inspect some of your predictions compared to the correct outputs, and describe some common types of errors in your report. What fraction of errors are due to failing to execute (e.g., syntax error), and what fraction are due to executing but getting the wrong answer?

In [ ]:
# Your code here
len(finetuned_predictions),len(splits['test'])
cnt=0
for i in range(len(finetuned_predictions)):
    if finetuned_predictions[i]!=splits['test'][i]['sql']:
        cnt+=1
        print(i,finetuned_predictions[i],"\n",splits['test'][i]['sql'])
print(cnt)

We can also examine the exact match accuracy (i.e., requiring the predicted SQL string to exactly match the gold answer) rather than the execution accuracy (just checking whether the output of executing the SQL against the database is the same).

In [ ]:
def check_exact_match_accuracy(predictions, data):
    assert len(predictions) == len(data)
    correct = 0
    for p, d in zip(predictions, data):
        if p == d['sql']:
            correct += 1
    return correct / len(predictions)

In [ ]:
print('finetuned exact match acc:', check_exact_match_accuracy(finetuned_predictions, splits['test']))

The exact match accuracy will likely be close to the execution accuracy, but not exactly the same. What are some potential pros and cons of each metric? Discuss in your report.

Unload your finetuned model so that you don't run out of GPU memory later.

In [ ]:
del finetuned_model

## Few-Shot Prompting

For the final part of this project, you will explore few-shot prompting, i.e., simply prompting the pretrained language model out-of-the-box using a small number of examples rather than finetuning.

First, let's try just selecting 4 examples completely at random from the training set. Rewrite your `predict_greedy` function to change the prompt:

In [ ]:
import random


few_shot_prompt = "Question: {question0}\n\nSQL: {sql0}\n\n\n\n" + \
    "Question: {question1}\n\nSQL: {sql1}\n\n\n\n" + \
    "Question: {question2}\n\nSQL: {sql2}\n\n\n\n" + \
    "Question: {question3}\n\nSQL: {sql3}\n\n\n\n" + \
    "Question: {question}\n\nSQL:"


def select_random_examples(question, few_shot_data, num_examples=4):
    """
    Return a list containing 4 of the elements of few_shot_data, selected randomly
    """
    return random.sample(few_shot_data, num_examples)


@torch.no_grad()
def predict_greedy_fewshot(model, data, few_shot_data, max_new_tokens=128, example_selection_method=select_random_examples):
    """
    Return the model's greedy text-to-sql predictions on the given data split.
    The maximum number of new tokens generated (NOT including tokens in the prompt) should be equal to max_new_tokens.
    The four examples with their SQL outputs should go in {question1}, {sql1}, {question2}, {sql2}, etc. in the few_shot_prompt. 
    The final {question} is the question that we're currently evaluating on.
    """
    questions = [d['question'] for d in data]
    predicted_sqls = []
    prompts = []
    for question in questions:
        few_shot_examples = example_selection_method(question, few_shot_data, num_examples=4)
        prompts.append(few_shot_prompt.format(
            question0=few_shot_examples[0]['question'],
            sql0=few_shot_examples[0]['sql'],
            question1=few_shot_examples[1]['question'],
            sql1=few_shot_examples[1]['sql'],
            question2=few_shot_examples[2]['question'],
            sql2=few_shot_examples[2]['sql'],
            question3=few_shot_examples[3]['question'],
            sql3=few_shot_examples[3]['sql'],
            question=question
        ))
    # Your code here; should be fairly similar to your previous predict_greedy code.
    # Hint: if you batch, we recommend batch size 8-16.
    batch_size=8
    tokenizer.padding_side='left'
    all_sqls=[]
    for i in range(0, len(data)):
        inputs = tokenizer(prompts[i], return_tensors="pt", padding=True).input_ids.cuda()
        output = model.generate(inputs, max_length=max_new_tokens, num_return_sequences=1, temperature=0,do_sample=False)
        generated_sqls = tokenizer.batch_decode(output, skip_special_tokens=True, max_length=max_new_tokens)
        generated_sqls_string=generated_sqls[0].split("\n")
        sql_strings= [s for s in generated_sqls_string if s.startswith('SQL: ')]
        all_sqls.append(sql_strings[4])

    predicted_sqls = [sql.replace("SQL:", "") for sql in all_sqls]
    return predicted_sqls # list of strings containing SQL predictions for each question in the data

In [ ]:
# Reload the gpt2 model if you need to
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2-medium').to(device) # Your code here

In [ ]:
# This call can take a few minutes even if you batch; you can debug on a subset of the dev set as needed.
predictions = predict_greedy_fewshot(gpt2_model, splits['test'], splits['train'])
print('4-shot prompting with random examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

You will probably observe between 0-5% accuracy. Random example selection doesn't work very well on this dataset. 

However, what if we select examples by picking the examples from the training set whose questions are most similar to our current question? To do this, load a pretrained sentence encoder, which takes a sentence as input and outputs a fixed-length vector encoding semantic information about that sentence. First compute the vectors associated with all the training set questions, and then select examples from the training set based on which question vectors have the largest dot products with the vector for your current question. 

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# Your code here; load the sentence encoder (see https://www.sbert.net/ for documentation). A good choice of model is "all-MiniLM-L6-v2"
sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2') 

def compute_question_encodings(data):
    """
    For each example in the data, add a field called 'question_encoding' to the example, which is the vector encoding of the question.
    """
    # Your code here
    vectors = sentence_encoder.encode([d['question'] for d in data],show_progress_bar=False)
    for example, vector in zip(data, vectors):
        example['question_encoding'] = np.array(vector)


def select_similar_examples(question, few_shot_data, num_examples=4):
    """
    Return a list containing 4 of the elements of few_shot_data, selected with questions most semantically similar to the given question. 
    The most similar question should be the LAST element of the list, second most similar should be the second to last element, etc.
    The reason is that in the few-shot prompt, **you want the best example to be the most recent one.**

    To rank by semantic similarity, first compute the vector for the current question, then compute its dot product with
    all training set vectors (hint: you may want to vectorize this computation using numpy). Then sort by dot product.

    You should take advantage of the 'question_encoding' field that you added to each example in compute_question_encodings.
    """
    # Your code here
    # hint: when you call .encode with your sentence encoder, use show_progress_bar=False to avoid tons of printouts
    current_question_encoding = sentence_encoder.encode([question], show_progress_bar=False)[0]
    similarities = np.dot(current_question_encoding, np.array([example['question_encoding'] for example in few_shot_data]).T)
    sorted_indices = np.argsort(similarities)[::-1]
    selected_examples = [few_shot_data[i] for i in sorted_indices[:num_examples]]
    
    return selected_examples


In [ ]:
# first precompute all the vectors for the training set
compute_question_encodings(splits['train'])

In [ ]:
# this call will again take a few minutes, even if you batched; feel free to debug on smaller sets of dev
predictions = predict_greedy_fewshot(gpt2_model, splits['test'], splits['train'], example_selection_method=select_similar_examples)
print('4-shot prompting with similar examples, execution acc:', check_execution_accuracy(predictions, splits['test']))

You should now achieve about 34% accuracy. The autograder will check that you get >30%.

Save your predictions.

In [ ]:
save_predictions(predictions, 'similar4shot_predictions.txt')

Once again, inspect some of your predictions (from prompting with similar examples) compared to the correct outputs, and describe some common types of errors in your report. Are there any differences compared to the finetuned model, or are the types of errors pretty similar? You can also check the exact match accuracy again.

Finally, do some open-ended exploration to try to improve your performance on this dataset as much as possible (whether for finetuning or prompting). No hard requirement on how much to improve (or to improve at all), but please discuss the ideas you tried + how effective they were in your report. (Be careful with the GPU memory if you're using Kaggle, though- we're already nearly capping out the GPU memory in a few places with the current settings.)

A non-exhaustive list of possible ideas:
* Use a different similarity metric for selecting examples in few-shot prompting
* Use more examples in few-shot prompting
* Load a different base model than GPT2-Medium, or look into calling the OpenAI API
* Tune the hyperparameters used for finetuning
* Try to combine few-shot prompting with finetuning

Your final submission should include the following files:

* hw4.ipynb (this file; please rename to match)
* finetuned_predictions.txt
* similar4shot_predictions.txt
* report.pdf